# 开始使用

## 结合poetry使用Jupyter

在poetry中支持jupyter和ipykernel：

In [ ]:
!poetry add ipykernel jupyter

创建一个名为`langchani-book-poetry-env`的可用内核：

In [ ]:
!poetry run python -m ipykernel install --user --name=langchani-book-poetry-env

## 加载环境变量

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

import sys
import io
original_stdout = sys.stdout 
sys.stdout = io.StringIO()
load_dotenv(find_dotenv(), override=True)
sys.stdout = original_stdout

## 使用langchain

## 使用langserve

## 访问langserve服务

<div class="alert alert-success">
<b>架构解偶：</b>在python中访问远程langserve的机制，有助于将GPT服务和应用模块在架构上解耦。
</div>

例如，要结合AI生成能力创建文档，下面这个封装会非常有用：

In [4]:
debug = True
base_url = "http://localhost:8000"

from IPython.display import display, Markdown
def print_content(data):
    for char in data:
        print(char, end="", flush=True)

from langserve import RemoteRunnable
def remote_runnable(runnable, params = [], base_url = base_url, debug = debug):
    chain = RemoteRunnable(f"{base_url}/{runnable}")
    def ask(*args):
        kwargs = dict(zip(params, args))
        if debug:
            print_content(chain.stream(kwargs))
        else:
            return chain.invoke(kwargs)
    return ask

或者打包为一个python文件，直接引用：

In [97]:
from utils import *

### GPT

In [5]:
gpt35 = remote_runnable("langserve/gpt35", ["question"])
gpt4 = remote_runnable("langserve/gpt4", ["question"])

In [93]:
gpt35("我的父母结婚时为什么没有邀请我？")

作为一个AI助手，我无法直接了解你父母结婚时为什么没有邀请你。这可能是因为他们有自己的原因和考虑。以下是一些可能的原因：

1. 仪式规模：有些婚礼可能只邀请亲近的家人和朋友，而不包括所有的亲属。这可能是为了控制婚礼的规模和预算。

2. 家庭关系：有时候，家庭关系可能复杂，可能存在紧张或冲突。父母可能希望避免潜在的问题或不愉快的场面，因此决定不邀请你。

3. 年龄限制：有些婚礼可能只邀请成年人或特定年龄段的亲戚和朋友。如果你当时还未成年，可能被排除在邀请范围之外。

4. 意外遗漏：有时候，邀请名单可能被意外遗漏或疏忽。这可能是一个错误，而不是有意排除你。

无论原因如何，如果你对此感到困惑或不满，最好与你的父母直接沟通，询问他们的想法和决定背后的原因。

In [94]:
gpt4("我的父母结婚时为什么没有邀请我？")

如果你的父母在结婚时没有邀请你，可能有几个原因：

1. 你还没有出生：最常见的原因是你在他们结婚时还没有出生。结婚是父母在你出生前的一个事件，所以你不可能被邀请。

2. 私密或小型婚礼：有些情况下，父母可能选择举行一个非常私密或小型的婚礼，只邀请非常亲近的家人和朋友。如果你当时年纪还小，他们可能认为你不适合参加这样的场合。

3. 地理或其他障碍：也可能是由于地理位置远、交通不便或其他实际障碍，你无法参加他们的婚礼。

4. 个人决定：每个家庭都有自己的动态和决策过程。可能有特定的原因导致他们决定不邀请你，这些原因可能涉及家庭关系、个人选择或当时的情况。

如果这个问题让你感到困惑或不安，与你的父母进行坦诚的对话可能是了解他们当时决定的最好方式。通常，这种情况下的解释很简单，就是你还没有出生。

### 通义千问

In [6]:
tongyi = remote_runnable("langserve/tongyi", ["question"])

In [7]:
tongyi("你是什么模型?")

我是阿里云开发的大规模语言模型，我叫通义千问。

In [87]:
tongyi("我的父母结婚时为什么没有邀请我?")

因为您在他们结婚的时候还没有出生，所以他们自然无法邀请您参加他们的婚礼。通常情况下，只有已经出生并能够理解事件意义的人才会被邀请参加婚礼。

### Chatglm3-6B

In [95]:
chatglm6b = remote_runnable("langserve/chatglm6b", ["question"])

In [96]:
chatglm6b("你是什么模型?")


 我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学KEG 实验室和智谱 AI公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。

In [88]:
chatglm6b("我的父母结婚时为什么没有邀请我?")


 作为人工智能助手，我无法了解您个人生活中的具体情况。但是，通常情况下，婚礼是一种庆祝两个人成为夫妇的仪式，因此，新婚夫妇很少会邀请孩子作为嘉宾。在某些情况下，他们可能会邀请孩子参加婚礼，但这取决于孩子的年龄和婚礼的规模。如果您希望了解更多关于您父母婚礼的信息，您可以尝试与他们沟通，了解他们当时的想法和决定。

### lambda函数

自定义的lambda函数一般不访问大模型，而是以较快速度在本地计算，因此不需要支持流式打印，所以无需封装。

In [5]:
chain = RemoteRunnable(f"{base_url}/langserve/add")
chain.invoke({"x": 1, "y": 2})

'The result is: 3'

In [6]:
chain = RemoteRunnable(f"{base_url}/langserve/add_one")
chain.invoke(42)

'The result is: 43'